## Code to perform several bioLQM-based pertubations

In [ ]:
import sys
import itertools
import biolqm
import ginsim
import seaborn as sns

#Load your model
gs_model = ginsim.load("Ypur_model.zginml")
lqm = ginsim.to_biolqm(gs_model)

### Define the perturbations of interest

In [ ]:
perturbations = ["AP_1%0", "Akt%0", "CASP8%0", "COX2%0", "CREB1%0", "DUSP%0", "EP1_Gq%0", "EP2_Gs%0", "EP3_Gi%0", "EP4_Gs%0", "ERK1_2%0", "FADD%0", "FOS%0", "IKBKG%0", "IKKb%0", "IRAK1%0", "IRAK4%0", "JNK%0", "JUN%0", "MEK%0", "MKK3_4%0", "MKK7%0", "MNK1%0", "MyD88%0", "NFKBIA%0", "NFkB%0", "PDPK1%0", "PGE2%0", "PIK3R1%0", "PKA%0", "PKC%0", "PLCb%0", "PROinfl_cyt%0", "RIPK1%0", "Res_Med%0", "TAB%0", "TAK1%0", "TLRs%0", "TNFAIP3%0", "TNFR1%0", "TRADD%0", "TRAF2%0", "TRAF6%0", "cFLIP%0", "cPLA2a%0", "p38%0"]

In [ ]:
# Perform double perturbations - All against all
all_pairwise = itertools.permutations(perturbations, 2)
all_pairwise_list = list(all_pairwise)
all_pairwise_list = str(all_pairwise_list).replace(',', ' ')
all_pairwise_list

In [ ]:
# Perform double perturbations - All nodes against one 
per = ["Node%0"] #define node that should be in combinatorial perturbations
Node_perturbations = list(itertools.product(per, perturbations))
Node_perturbations = str(Node_perturbations).replace(',', ' ')
Node_perturbations

### Perform pairwise mutations

In [ ]:
'''Define dictionaries to store attractors for each perturbation'''
fixpointlist = {}

'''trapspacelist is used for perturbations where no stable state is found
    to find eventual cyclic attractors.'''
trapspacelist = {}

for p in perturbations:
    fixpoints = biolqm.fixpoints(biolqm.perturbation(lqm, p))
    if(fixpoints):
        fixpointlist[p] = fixpoints
    else:
        '''No stable state found, so we look for cycles'''
        trapspace = biolqm.trapspace(biolqm.perturbation(lqm, p))
        trapspacelist[p] = trapspace

#see attractors in tabulated tables for each perturbation
for k, v in fixpointlist.items():
    print(k)
    sys.displayhook(tabulate(v))
    print()
    
for k, v in trapspacelist.items():
    print(k)
    sys.displayhook(tabulate(v))
    print()

In [ ]:
'''List of perturbations with the nb of stable states '''
perturbstates=[]
trappedstates=[]

'''List of table state values same order as perturbStates'''
stablestates = [] 
tstates = [] 

'''List of nodes'''
nodelist = ['TNFRSF1A', 'TRAF6', 'TAK1', 'IKBKB', 'NFKBIA', 'NFKBIZ', 'IKBKG', 'NFKB', 'TNFAIP3', 'P38', 'ERK1_2', 'JNK', 'MNK1', 'PIK3R1', 'PDPK1', 'AKT', 'PRKCA', 'AP1', 'CREB1', 'PLCG1', 'PRKACA', 'RIPK1', 'TRADD', 'TRAF2', 'FADD', 'CASP8', 'CFLAR', 'IL8', 'IL6', 'BAD', 'BCL2', 'CDKN1A', 'CCND1', 'IFNGR', 'STAT1', 'IL36', 'IL17R', 'CEBP', 'IL19', 'IL22R', 'STAT3', 'SOCS3', 'SOCS1', 'SIRT1', 'AA', 'cPLA2a', 'PTGS2', 'PGE2', 'EP1', 'EP2', 'EP3', 'EP4', 'PGI2', 'EGR1', 'ALOX5', 'LTB4', 'SP1', 'ALOX12', 'HETE12', 'PPARD', 'ALOX15', 'HETE15', 'TRAF3IP2', 'KRT1', 'FLG', 'CALML5', 'CXCL3', 'IFNG', 'IL12', 'TNFa', 'IL1B', 'CCL2', 'CCL5', 'CXCR3', 'CCL20', 'CXCL1', 'CXCL2', 'CXCL5', 'EGFR', 'VDR', 'EP2_g', 'EP4_g', 'S100A7', 'S100A8', 'S100A9', 'DEFB4A', 'HBEGF', 'CSF3', 'CCL7', 'FOXO3']
for k,v in fixpointlist.items():   
    for i in range(0,len(v)):
        listSS = []
        for key, val in v[i].items():
            listSS.append(val)
        stablestates.append(listSS)
        perturbstates.append(str(k)+' '+str(i))

'''Create dataframe: columns = nodes, rows = perturbations'''
df = DataFrame(data=stablestates)
df.columns = nodelist
df.index = perturbstates

'''Generate clustered heatmap'''
cm = sns.clustermap(df, metric='euclidean', cmap = 'Blues',  figsize=(15, 25))
#cm.savefig("SSclusters.png")